In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from fim import apriori
sys.path.insert(0, '../../Daniele')
sys.path.insert(0, '../Part1')
sys.path.insert(0, '../../Riccardo')

from MissingValues_3 import remove_missing_values
from outliers import countOutliers
from outliers import removeOutliers

In [2]:
#load dataset into a dataframe
data = pd.read_csv("../../../Dataset/credit_default_train.csv")#credit_default_cleaned.csv")
data = remove_missing_values(data)
removeOutliers(data)

Initial size of data frame:  (10000, 24)
Visual analysis, number of rows to be dropped:  30
Final size of data frame:  (9970, 24)


In [3]:
data['AgeBin'] = pd.cut(data['age'].astype(int), 10, right=False)
data['LimitBin'] = pd.cut(data['limit'].astype(int), 7, right=False)

data.drop(['age', 'limit'], axis=1, inplace=True)

In [4]:
data.head()

,sex,education,status,ps-sep,ps-aug,ps-jul,ps-jun,ps-may,ps-apr,ba-sep,...,ba-apr,pa-sep,pa-aug,pa-jul,pa-jun,pa-may,pa-apr,credit_default,AgeBin,LimitBin
0,male,graduate school,single,2,0,0,0,0,0,48966,...,30302,2130,1905,1811,1100,1100,1200,no,"[21.0, 26.4)","[10000.0, 120000.0)"
1,male,university,married,-1,-1,-1,-1,-1,-1,10755,...,7956,10120,7852,6336,4622,7956,5499,no,"[53.4, 58.8)","[120000.0, 230000.0)"
2,female,high school,married,2,3,2,2,2,0,30057,...,29047,0,1700,1100,3,1053,1303,no,"[37.2, 42.6)","[10000.0, 120000.0)"
3,female,university,single,0,0,0,0,0,0,56426,...,23783,2000,2000,900,2000,10000,5000,no,"[26.4, 31.8)","[120000.0, 230000.0)"
4,female,high school,married,1,2,2,0,0,0,47987,...,52773,1788,0,1894,1801,3997,0,yes,"[31.8, 37.2)","[10000.0, 120000.0)"


In [5]:
data['AgeBin'].unique()

[[21.0, 26.4), [53.4, 58.8), [37.2, 42.6), [26.4, 31.8), [31.8, 37.2), [42.6, 48.0), [48.0, 53.4), [58.8, 64.2), [69.6, 75.054), [64.2, 69.6)]
Categories (10, interval[float64]): [[21.0, 26.4) < [26.4, 31.8) < [31.8, 37.2) < [37.2, 42.6) ... [53.4, 58.8) < [58.8, 64.2) < [64.2, 69.6) < [69.6, 75.054)]

In [6]:
data['ba-sep_bin'] = pd.cut(data['ba-sep'].astype(int), 7, right=False)
data['ba-aug_bin'] = pd.cut(data['ba-aug'].astype(int), 7, right=False)
data['ba-jul_bin'] = pd.cut(data['ba-jul'].astype(int), 7, right=False)
data['ba-jun_bin'] = pd.cut(data['ba-jun'].astype(int), 7, right=False)
data['ba-may_bin'] = pd.cut(data['ba-may'].astype(int), 7, right=False)
data['ba-apr_bin'] = pd.cut(data['ba-apr'].astype(int), 7, right=False)

In [7]:
data.drop(['ba-sep', 'ba-aug', 'ba-jul', 'ba-jun', 'ba-may', 'ba-apr'], axis=1, inplace=True)

In [8]:
data.head()

,sex,education,status,ps-sep,ps-aug,ps-jul,ps-jun,ps-may,ps-apr,pa-sep,...,pa-apr,credit_default,AgeBin,LimitBin,ba-sep_bin,ba-aug_bin,ba-jul_bin,ba-jun_bin,ba-may_bin,ba-apr_bin
0,male,graduate school,single,2,0,0,0,0,0,2130,...,1200,no,"[21.0, 26.4)","[10000.0, 120000.0)","[-14386.0, 75363.429)","[26754.429, 123285.857)","[32625.571, 126757.143)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)"
1,male,university,married,-1,-1,-1,-1,-1,-1,10120,...,5499,no,"[53.4, 58.8)","[120000.0, 230000.0)","[-14386.0, 75363.429)","[-69777.0, 26754.429)","[-61506.0, 32625.571)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)"
2,female,high school,married,2,3,2,2,2,0,0,...,1303,no,"[37.2, 42.6)","[10000.0, 120000.0)","[-14386.0, 75363.429)","[26754.429, 123285.857)","[-61506.0, 32625.571)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)"
3,female,university,single,0,0,0,0,0,0,2000,...,5000,no,"[26.4, 31.8)","[120000.0, 230000.0)","[-14386.0, 75363.429)","[26754.429, 123285.857)","[32625.571, 126757.143)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)"
4,female,high school,married,1,2,2,0,0,0,1788,...,0,yes,"[31.8, 37.2)","[10000.0, 120000.0)","[-14386.0, 75363.429)","[26754.429, 123285.857)","[32625.571, 126757.143)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)"


In [9]:
data['pa-sep_bin'] = pd.cut(data['pa-sep'].astype(int), 5, right=False)
data['pa-aug_bin'] = pd.cut(data['pa-aug'].astype(int), 5, right=False)
data['pa-jul_bin'] = pd.cut(data['pa-jul'].astype(int), 5, right=False)
data['pa-jun_bin'] = pd.cut(data['pa-jun'].astype(int), 5, right=False)
data['pa-may_bin'] = pd.cut(data['pa-may'].astype(int), 5, right=False)
data['pa-apr_bin'] = pd.cut(data['pa-apr'].astype(int), 5, right=False)

In [10]:
data.drop(['pa-sep', 'pa-aug', 'pa-jul', 'pa-jun', 'pa-may', 'pa-apr'], axis=1, inplace=True)

In [11]:
data.head()

,sex,education,status,ps-sep,ps-aug,ps-jul,ps-jun,ps-may,ps-apr,credit_default,...,ba-jul_bin,ba-jun_bin,ba-may_bin,ba-apr_bin,pa-sep_bin,pa-aug_bin,pa-jul_bin,pa-jun_bin,pa-may_bin,pa-apr_bin
0,male,graduate school,single,2,0,0,0,0,0,no,...,"[32625.571, 126757.143)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)","[0.0, 64602.8)","[0.0, 77625.2)","[0.0, 83517.6)","[0.0, 58592.4)","[0.0, 60702.4)","[0.0, 88600.2)"
1,male,university,married,-1,-1,-1,-1,-1,-1,no,...,"[-61506.0, 32625.571)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)","[0.0, 64602.8)","[0.0, 77625.2)","[0.0, 83517.6)","[0.0, 58592.4)","[0.0, 60702.4)","[0.0, 88600.2)"
2,female,high school,married,2,3,2,2,2,0,no,...,"[-61506.0, 32625.571)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)","[0.0, 64602.8)","[0.0, 77625.2)","[0.0, 83517.6)","[0.0, 58592.4)","[0.0, 60702.4)","[0.0, 88600.2)"
3,female,university,single,0,0,0,0,0,0,no,...,"[32625.571, 126757.143)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)","[0.0, 64602.8)","[0.0, 77625.2)","[0.0, 83517.6)","[0.0, 58592.4)","[0.0, 60702.4)","[0.0, 88600.2)"
4,female,high school,married,1,2,2,0,0,0,yes,...,"[32625.571, 126757.143)","[-24303.0, 67288.286)","[-4291.0, 80188.714)","[-4543.0, 77340.0)","[0.0, 64602.8)","[0.0, 77625.2)","[0.0, 83517.6)","[0.0, 58592.4)","[0.0, 60702.4)","[0.0, 88600.2)"


In [12]:
data.columns

Index(['sex', 'education', 'status', 'ps-sep', 'ps-aug', 'ps-jul', 'ps-jun',
       'ps-may', 'ps-apr', 'credit_default', 'AgeBin', 'LimitBin',
       'ba-sep_bin', 'ba-aug_bin', 'ba-jul_bin', 'ba-jun_bin', 'ba-may_bin',
       'ba-apr_bin', 'pa-sep_bin', 'pa-aug_bin', 'pa-jul_bin', 'pa-jun_bin',
       'pa-may_bin', 'pa-apr_bin'],
      dtype='object')

In [13]:
data['ba-sep_bin'] = data['ba-sep_bin'].astype(str) + '_ba-sep'
data['ba-aug_bin'] = data['ba-aug_bin'].astype(str) + '_ba-aug'
data['ba-jul_bin'] = data['ba-jul_bin'].astype(str) + '_ba-jul'
data['ba-jun_bin'] = data['ba-jun_bin'].astype(str) + '_ba-jun'
data['ba-may_bin'] = data['ba-may_bin'].astype(str) + '_ba-may'
data['ba-apr_bin'] = data['ba-apr_bin'].astype(str) + '_ba-apr'

data['pa-sep_bin'] = data['pa-sep_bin'].astype(str) + '_pa-sep'
data['pa-aug_bin'] = data['pa-aug_bin'].astype(str) + '_pa-aug'
data['pa-jul_bin'] = data['pa-jul_bin'].astype(str) + '_pa-jul'
data['pa-jun_bin'] = data['pa-jun_bin'].astype(str) + '_pa-jun'
data['pa-may_bin'] = data['pa-may_bin'].astype(str) + '_pa-may'
data['pa-apr_bin'] = data['pa-apr_bin'].astype(str) + '_pa-apr'

data['AgeBin'] = data['AgeBin'].astype(str) + '_Age'
data['LimitBin'] = data['LimitBin'].astype(str) + '_Limit'

In [14]:
data['ps-sep'] = data['ps-sep'].astype(str) + '_ps-sep'
data['ps-aug'] = data['ps-aug'].astype(str) + '_ps-aug'
data['ps-jul'] = data['ps-jul'].astype(str) + '_ps-jul'
data['ps-jun'] = data['ps-jun'].astype(str) + '_ps-jun'
data['ps-may'] = data['ps-may'].astype(str) + '_ps-may'
data['ps-apr'] = data['ps-apr'].astype(str) + '_ps-apr'

In [15]:
data.head()

,sex,education,status,ps-sep,ps-aug,ps-jul,ps-jun,ps-may,ps-apr,credit_default,...,ba-jul_bin,ba-jun_bin,ba-may_bin,ba-apr_bin,pa-sep_bin,pa-aug_bin,pa-jul_bin,pa-jun_bin,pa-may_bin,pa-apr_bin
0,male,graduate school,single,2_ps-sep,0_ps-aug,0_ps-jul,0_ps-jun,0_ps-may,0_ps-apr,no,...,"[32625.571, 126757.143)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
1,male,university,married,-1_ps-sep,-1_ps-aug,-1_ps-jul,-1_ps-jun,-1_ps-may,-1_ps-apr,no,...,"[-61506.0, 32625.571)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
2,female,high school,married,2_ps-sep,3_ps-aug,2_ps-jul,2_ps-jun,2_ps-may,0_ps-apr,no,...,"[-61506.0, 32625.571)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
3,female,university,single,0_ps-sep,0_ps-aug,0_ps-jul,0_ps-jun,0_ps-may,0_ps-apr,no,...,"[32625.571, 126757.143)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
4,female,high school,married,1_ps-sep,2_ps-aug,2_ps-jul,0_ps-jun,0_ps-may,0_ps-apr,yes,...,"[32625.571, 126757.143)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"


In [16]:
cd_ = data['credit_default']
data = data.drop(columns=['credit_default'])
data.insert(loc=0, column='credit_default', value=cd_)
data.head()

,credit_default,sex,education,status,ps-sep,ps-aug,ps-jul,ps-jun,ps-may,ps-apr,...,ba-jul_bin,ba-jun_bin,ba-may_bin,ba-apr_bin,pa-sep_bin,pa-aug_bin,pa-jul_bin,pa-jun_bin,pa-may_bin,pa-apr_bin
0,no,male,graduate school,single,2_ps-sep,0_ps-aug,0_ps-jul,0_ps-jun,0_ps-may,0_ps-apr,...,"[32625.571, 126757.143)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
1,no,male,university,married,-1_ps-sep,-1_ps-aug,-1_ps-jul,-1_ps-jun,-1_ps-may,-1_ps-apr,...,"[-61506.0, 32625.571)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
2,no,female,high school,married,2_ps-sep,3_ps-aug,2_ps-jul,2_ps-jun,2_ps-may,0_ps-apr,...,"[-61506.0, 32625.571)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
3,no,female,university,single,0_ps-sep,0_ps-aug,0_ps-jul,0_ps-jun,0_ps-may,0_ps-apr,...,"[32625.571, 126757.143)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"
4,yes,female,high school,married,1_ps-sep,2_ps-aug,2_ps-jul,0_ps-jun,0_ps-may,0_ps-apr,...,"[32625.571, 126757.143)_ba-jul","[-24303.0, 67288.286)_ba-jun","[-4291.0, 80188.714)_ba-may","[-4543.0, 77340.0)_ba-apr","[0.0, 64602.8)_pa-sep","[0.0, 77625.2)_pa-aug","[0.0, 83517.6)_pa-jul","[0.0, 58592.4)_pa-jun","[0.0, 60702.4)_pa-may","[0.0, 88600.2)_pa-apr"


In [17]:
baskets = data.values.tolist()

In [18]:
baskets[0]

['no',
 'male',
 'graduate school',
 'single',
 '2_ps-sep',
 '0_ps-aug',
 '0_ps-jul',
 '0_ps-jun',
 '0_ps-may',
 '0_ps-apr',
 '[21.0, 26.4)_Age',
 '[10000.0, 120000.0)_Limit',
 '[-14386.0, 75363.429)_ba-sep',
 '[26754.429, 123285.857)_ba-aug',
 '[32625.571, 126757.143)_ba-jul',
 '[-24303.0, 67288.286)_ba-jun',
 '[-4291.0, 80188.714)_ba-may',
 '[-4543.0, 77340.0)_ba-apr',
 '[0.0, 64602.8)_pa-sep',
 '[0.0, 77625.2)_pa-aug',
 '[0.0, 83517.6)_pa-jul',
 '[0.0, 58592.4)_pa-jun',
 '[0.0, 60702.4)_pa-may',
 '[0.0, 88600.2)_pa-apr']

In [19]:
#rules = apriori(baskets, supp=10, zmin=2, conf=60, target='r', report='scl') 

In [ ]:
for s in range(1, 100):
    #for c in range(1, 100):
    print("starting apriori")
    rules = apriori(baskets, supp=s, zmin=2, target='a', conf=60, report='scl')
    print("ended apriori")
    numRules= len(rules)
    print(numRules)
    if (numRules == 0):
        break
    print('~~~~~~~~~Supp: ', s, ' Conf: 60 Number of rules:', len(rules))
    for r in rules:
        if r[0] == 'yes':
            print(r)